## Collect demographic data

age, sex, language, marital_status, race

In [ ]:
import gzip
import pandas as pd
import gc

In [ ]:
# Read the exported for rows chartevent CSV file
choosen_patients_df = pd.read_csv('CSV\Exports\o03_icu_first_stay.csv')

# Read files
compressed_patients_df = r"..\Datasets\mimic-iv-2_2\hosp\patients.csv.gz"
compressed_admissions_df = r"..\Datasets\mimic-iv-2_2\hosp\admissions.csv.gz"

# Read and pass the compressed CSV file into a DataFrame
patients_df = pd.read_csv(compressed_patients_df, compression='gzip')
admissions_df = pd.read_csv(compressed_admissions_df, compression='gzip')

In [ ]:
# Age calculation

# Convert the 'intime' column to datetime and then extract the year
choosen_patients_df['intime'] = pd.to_datetime(choosen_patients_df['intime'])
choosen_patients_df['year'] = choosen_patients_df['intime'].dt.year

# Choosing only the subject_id and year
choosen_patients_df = choosen_patients_df[['subject_id','hadm_id','year']]

# Merge the tables
age_df = pd.merge(choosen_patients_df, patients_df, on='subject_id')

# Calculate the actual age based on the conditions
age_df['age'] = age_df.apply(lambda row: min(91, row['anchor_age'] + (row['year'] - row['anchor_year'])), axis=1)

# Keep subject, gender and age at the ICU admission.
age_df = age_df[['subject_id', 'gender', 'age']]

In [ ]:
# Keep language, marital status, race and expire flag(live or death) 
demographics_df = pd.merge(choosen_patients_df, admissions_df, on=['subject_id', 'hadm_id'])
demographics_df = demographics_df[['subject_id', 'hadm_id', 'language', 'marital_status', 'race', 'hospital_expire_flag']]

In [ ]:
temp_df = pd.merge(age_df, demographics_df, on=['subject_id'])

# Set new order
new_order = ['subject_id', 'hadm_id'] + [col for col in temp_df.columns if col not in ['subject_id', 'hadm_id']]

# Reorder the columns in the DataFrame
temp_df = temp_df[new_order]

# Export the merged DataFrame to a CSV file
temp_df.to_csv('CSV\Exports\o08_demographics.csv', index=False)

# Free RAM
gc.collect()